In [1]:
import os
import openai
import json
import gradio as gr

import asyncio
import nest_asyncio

from agents import Agent, Runner, function_tool, SQLiteSession
from openai import OpenAI

# Pour GoogleColab
# from google.colab import userdata 
# Hors GoogleColab 3 lignes ci-dessous
from dotenv import load_dotenv
load_dotenv(override=True)

nest_asyncio.apply()

# openai_api_key = userdata.get('OPENAI_API_KEY')  # Pour GoogleColab
openai_api_key = os.getenv('OPENAI_API_KEY')     # Hors GoogleColab 
client = OpenAI(api_key=openai_api_key)
os.environ["OPENAI_API_KEY"] = openai_api_key

system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

@function_tool
def get_ticket_price(destination_city: str):
   """ Get the price of a return ticket to the given destination city """
   print(f"Tool get_ticket_price called for city {destination_city}")
   price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
   return f"The price of a ticket to {destination_city} is {price}"

city_weather = {"london": "21°", "paris": "22°", "tokyo": "15°", "berlin": "14°"}

@function_tool
def get_weather(destination_city: str):
   """ Get the temperature of the given destination city """
   print(f"Tool get_weather called for city {destination_city}")
   temperature = city_weather.get(destination_city.lower(), "Unknown city")
   return f"The temperature in {destination_city} is {temperature}"

tools = [get_ticket_price, get_weather]

agent = Agent(
   name="Assistant",
   instructions=system_message,
   model="gpt-4o-mini-2024-07-18",
   tools=tools)

session = SQLiteSession("conversation_1")

async def chat_async(message: str, session=session):
   result = await Runner.run(agent, message, session=session)
   return result.final_output

def chat(message: str, session=session):
   loop = asyncio.get_event_loop()
   return loop.run_until_complete(chat_async(message, session))

print("Configuration terminée!")


/home/matos/projects/tests_ia_agentique/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configuration terminée!


In [ ]:
# Conversation avec utilisation de Gradio
def chat_gradio(message, history):
    return chat(message, session)

gr.ChatInterface(
    fn=chat_gradio,
    title="✈️ FlightAI Assistant",
    description="Ask me about ticket prices and weather!",
    examples=[
        "What's the price to Paris?",
        "What's the weather in Tokyo?",
        "I want to go to London"
    ]
).launch(debug=True)

In [ ]:
# Conversation version ligne de commande 
print("Chat démarré (tapez 'exit' pour quitter)\n")

while True:
    try:
        user_input = input("Vous > ").strip()
        
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Au revoir !")
            break
        
        if not user_input:
            continue
        
        bot_response = chat(user_input, session=session)
        print(f"Bot  > {bot_response}\n")
        
    except KeyboardInterrupt:
        print("\n\nAu revoir !")
        break